## App directories

In [1]:
from pathlib import Path
import subprocess, time, shutil
from datetime import datetime

def sh_stream(cmd, cwd=None, timeout=1800):
    """Run a shell command and stream output live in Jupyter."""
    print(">", cmd)
    p = subprocess.Popen(
        cmd, cwd=cwd, shell=True,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    start = time.time()
    for line in p.stdout:
        print(line, end="")
        if time.time() - start > timeout:
            p.kill()
            raise TimeoutError(f"Timeout after {timeout}s: {cmd}")
    rc = p.wait()
    if rc != 0:
        raise RuntimeError(f"Command failed ({rc}): {cmd}")


In [2]:
# Repo A root (run notebook from inside agentic-orchestrator)
ORCH_DIR = Path.cwd()

# Workspace folder (ignored by Repo A)
SANDBOX_DIR = ORCH_DIR / "sandbox"
SANDBOX_DIR.mkdir(parents=True, exist_ok=True)

# Static repo name provided by user (Repo B remote)
repo_name = "personal-ai-planner"   # <-- change as needed

# Each run gets a fresh local folder
run_tag = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = SANDBOX_DIR / f"run_{run_tag}"
run_dir.mkdir(parents=True, exist_ok=False)

print("Repo A (orchestrator):", ORCH_DIR)
print("Sandbox:", SANDBOX_DIR)
print("This run dir:", run_dir)
print("Repo B name (static):", repo_name)


Repo A (orchestrator): c:\PROJECTS\agentic-demo\agentic-orchestrator
Sandbox: c:\PROJECTS\agentic-demo\agentic-orchestrator\sandbox
This run dir: c:\PROJECTS\agentic-demo\agentic-orchestrator\sandbox\run_20260110_221722
Repo B name (static): personal-ai-planner


In [3]:
sh_stream("gh --version")
sh_stream("gh auth status")
sh_stream("gh auth setup-git")


> gh --version
gh version 2.83.2 (2025-12-10)
https://github.com/cli/cli/releases/tag/v2.83.2
> gh auth status
github.com
  âœ“ Logged in to github.com account Iguta (keyring)
  - Active account: true
  - Git operations protocol: https
  - Token: gho_************************************
  - Token scopes: 'gist', 'read:org', 'repo', 'workflow'
> gh auth setup-git


In [4]:
def repo_exists(name: str) -> bool:
    p = subprocess.run(
        f'gh repo view {name} --json name -q .name',
        shell=True, capture_output=True, text=True
    )
    return p.returncode == 0

if not repo_exists(repo_name):
    visibility = "--public"  # or "--private"
    sh_stream(f"gh repo create {repo_name} {visibility} --confirm", cwd=run_dir)
    print("Created repo:", repo_name)
else:
    print("Repo exists, reusing:", repo_name)


Repo exists, reusing: personal-ai-planner


In [5]:
https_clone = subprocess.check_output(
    f'gh repo view {repo_name} --json httpsUrl -q .httpsUrl',
    shell=True, text=True
).strip()

print("HTTPS clone URL:", https_clone)


CalledProcessError: Command 'gh repo view personal-ai-planner --json httpsUrl -q .httpsUrl' returned non-zero exit status 1.

In [7]:
OWNER = "Iguta"
https_clone = f"https://github.com/{OWNER}/{repo_name}.git"
print("HTTPS clone URL:", https_clone)


HTTPS clone URL: https://github.com/Iguta/personal-ai-planner.git


In [9]:
# Clone into the empty run_dir
sh_stream(f'git clone "{https_clone}" .', cwd=run_dir)

# Create main branch explicitly (repo is empty)
sh_stream("git checkout -b main", cwd=run_dir)

# Commit scaffolded files
sh_stream("git add .", cwd=run_dir)
sh_stream('git commit -m "chore: initial scaffold"', cwd=run_dir)

# Push and set upstream
sh_stream("git push -u origin main", cwd=run_dir)

# Ensure on main and up-to-date
sh_stream("git checkout main", cwd=run_dir)
sh_stream("git pull", cwd=run_dir)

# Sanity
sh_stream("git remote -v", cwd=run_dir)
sh_stream("git status", cwd=run_dir)


> git clone "https://github.com/Iguta/personal-ai-planner.git" .
fatal: destination path '.' already exists and is not an empty directory.


RuntimeError: Command failed (128): git clone "https://github.com/Iguta/personal-ai-planner.git" .